# RAN Database Data Visualization and Analysis

This notebook provides comprehensive visualization and analysis of the imported RAN performance data.

## Overview
- **Purpose**: Analyze imported RAN 2G/4G performance data
- **Data Source**: SQLite database with imported CSV data
- **Analysis Types**: Statistical, temporal, geographic, and performance analysis

---

## 1. Import Required Libraries

Import all necessary libraries for data analysis and visualization.

In [ ]:
# Core data manipulation and analysis
import pandas as pd
import numpy as np
import sqlite3
from pathlib import Path
import sys
import warnings
warnings.filterwarnings('ignore')

# Visualization libraries
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.offline as pyo

# Statistical analysis
from scipy import stats
from scipy.stats import pearsonr, spearmanr

# Date and time handling
from datetime import datetime, timedelta
import dateutil.parser

# Setup plotting style
plt.style.use('default')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10

# Enable interactive plots in Jupyter
pyo.init_notebook_mode(connected=True)

# Add project root to path
PROJECT_ROOT = Path().absolute().parent.parent
sys.path.insert(0, str(PROJECT_ROOT))

print("📚 Libraries imported successfully!")
print(f"🏠 Project root: {PROJECT_ROOT}")

## 2. Database Connection Setup

Establish connection to the SQLite database and create helper functions.

In [ ]:
# Database configuration
DATABASE_PATH = PROJECT_ROOT / "data" / "databases" / "ran_performance.db"

class RANDatabaseAnalyzer:
    def __init__(self, db_path):
        self.db_path = db_path
        
    def get_connection(self):
        """Get database connection."""
        return sqlite3.connect(self.db_path)
    
    def execute_query(self, query, params=None):
        """Execute a query and return results as DataFrame."""
        with self.get_connection() as conn:
            return pd.read_sql_query(query, conn, params=params)
    
    def get_table_list(self):
        """Get list of all tables in the database."""
        query = "SELECT name FROM sqlite_master WHERE type='table'"
        return self.execute_query(query)['name'].tolist()
    
    def get_table_info(self, table_name):
        """Get detailed information about a table."""
        # Get column info
        pragma_query = f"PRAGMA table_info({table_name})"
        columns_df = self.execute_query(pragma_query)
        
        # Get row count
        count_query = f"SELECT COUNT(*) as row_count FROM {table_name}"
        row_count = self.execute_query(count_query)['row_count'].iloc[0]
        
        return {
            'table_name': table_name,
            'row_count': row_count,
            'columns': columns_df
        }
    
    def sample_data(self, table_name, n=1000):
        """Get a random sample of data from a table."""
        query = f"SELECT * FROM {table_name} ORDER BY RANDOM() LIMIT {n}"
        return self.execute_query(query)

# Initialize analyzer
if DATABASE_PATH.exists():
    analyzer = RANDatabaseAnalyzer(DATABASE_PATH)
    print(f"✅ Connected to database: {DATABASE_PATH}")
else:
    print(f"❌ Database not found: {DATABASE_PATH}")
    print("💡 Run the import script first to create the database.")

## 3. Database Schema Overview

Display all tables, their schemas, and basic statistics.

In [ ]:
# Get all tables
tables = analyzer.get_table_list()
print(f"📊 Found {len(tables)} tables in the database:")

# Display table information
table_summary = []
for table in tables:
    info = analyzer.get_table_info(table)
    table_summary.append({
        'Table': table,
        'Rows': f"{info['row_count']:,}",
        'Columns': len(info['columns']),
        'Technology': '2G' if '2g' in table.lower() else '4G' if '4g' in table.lower() else 'Unknown'
    })
    print(f"   📋 {table}: {info['row_count']:,} rows, {len(info['columns'])} columns")

# Create summary DataFrame
summary_df = pd.DataFrame(table_summary)
print("\n📈 Database Summary:")
display(summary_df)

# Total rows across all tables
total_rows = sum([int(row['Rows'].replace(',', '')) for row in table_summary])
print(f"\n🎯 Total rows across all tables: {total_rows:,}")

In [ ]:
# Detailed schema for each table
print("🔍 Detailed Schema Information:")

for table in tables[:2]:  # Show first 2 tables to avoid clutter
    print(f"\n📋 Table: {table}")
    info = analyzer.get_table_info(table)
    columns_df = info['columns']
    
    # Display column information
    display(columns_df[['name', 'type', 'notnull', 'dflt_value']].head(10))
    
    if len(columns_df) > 10:
        print(f"   ... and {len(columns_df) - 10} more columns")

## 4. Data Summary Statistics

Generate descriptive statistics for numerical columns.

In [ ]:
# Get sample data from each table for analysis
sample_data = {}
for table in tables:
    try:
        sample_data[table] = analyzer.sample_data(table, n=5000)
        print(f"✅ Loaded {len(sample_data[table])} sample rows from {table}")
    except Exception as e:
        print(f"❌ Error loading {table}: {e}")

print(f"\n📊 Sample data loaded from {len(sample_data)} tables")

In [ ]:
# Generate descriptive statistics for each table
for table_name, df in sample_data.items():
    print(f"\n📈 Descriptive Statistics for {table_name}:")
    print(f"Shape: {df.shape}")
    
    # Get numerical columns
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    
    if len(numeric_cols) > 0:
        stats_df = df[numeric_cols].describe()
        display(stats_df.round(2))
        
        # Missing values
        missing_data = df.isnull().sum()
        missing_pct = (missing_data / len(df)) * 100
        missing_summary = pd.DataFrame({
            'Missing Count': missing_data,
            'Missing %': missing_pct.round(2)
        }).query('`Missing Count` > 0')
        
        if not missing_summary.empty:
            print("\n🚨 Missing Data Summary:")
            display(missing_summary)
    else:
        print("No numerical columns found.")

## 5. Technology Distribution Analysis

Analyze the distribution of different RAN technologies in the dataset.

In [ ]:
# Technology distribution across tables
tech_distribution = {}
for table_name, df in sample_data.items():
    if '2g' in table_name.lower():
        tech_distribution['2G'] = tech_distribution.get('2G', 0) + len(df)
    elif '4g' in table_name.lower():
        tech_distribution['4G'] = tech_distribution.get('4G', 0) + len(df)
    else:
        tech_distribution['Other'] = tech_distribution.get('Other', 0) + len(df)

# Create pie chart for technology distribution
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Pie chart
colors = ['#FF6B6B', '#4ECDC4', '#45B7D1', '#96CEB4', '#FFEAA7']
wedges, texts, autotexts = ax1.pie(tech_distribution.values(), 
                                  labels=tech_distribution.keys(),
                                  autopct='%1.1f%%',
                                  colors=colors)
ax1.set_title('📡 RAN Technology Distribution (Sample Data)', fontsize=14, fontweight='bold')

# Bar chart
bars = ax2.bar(tech_distribution.keys(), tech_distribution.values(), color=colors)
ax2.set_title('📊 Sample Data Count by Technology', fontsize=14, fontweight='bold')
ax2.set_ylabel('Number of Records')
ax2.set_xlabel('Technology')

# Add value labels on bars
for bar in bars:
    height = bar.get_height()
    ax2.annotate(f'{int(height):,}',
                xy=(bar.get_x() + bar.get_width() / 2, height),
                xytext=(0, 3),
                textcoords="offset points",
                ha='center', va='bottom')

plt.tight_layout()
plt.show()

print("\n📋 Technology Distribution Summary:")
for tech, count in tech_distribution.items():
    print(f"   {tech}: {count:,} records ({count/sum(tech_distribution.values())*100:.1f}%)")

## 6. Performance Metrics Visualization

Analyze key performance indicators across different technologies.

In [ ]:
# Identify common performance metrics columns
performance_keywords = ['throughput', 'latency', 'signal', 'rssi', 'rsrp', 'rsrq', 'sinr', 
                       'bler', 'utilization', 'availability', 'quality', 'strength']

perf_columns = {}
for table_name, df in sample_data.items():
    cols = [col for col in df.columns if any(keyword in col.lower() for keyword in performance_keywords)]
    numeric_cols = df[cols].select_dtypes(include=[np.number]).columns.tolist()
    if numeric_cols:
        perf_columns[table_name] = numeric_cols

print("🎯 Identified Performance Metrics:")
for table, cols in perf_columns.items():
    print(f"   {table}: {len(cols)} metrics - {cols[:3]}{'...' if len(cols) > 3 else ''}")

In [ ]:
# Create performance metrics visualization
if perf_columns:
    # Select first table with performance metrics
    main_table = list(perf_columns.keys())[0]
    main_df = sample_data[main_table]
    main_cols = perf_columns[main_table][:6]  # First 6 metrics
    
    if main_cols:
        fig, axes = plt.subplots(2, 3, figsize=(18, 12))
        axes = axes.flatten()
        
        for i, col in enumerate(main_cols):
            if i < len(axes):
                ax = axes[i]
                
                # Remove outliers for better visualization
                data = main_df[col].dropna()
                if len(data) > 0:
                    Q1 = data.quantile(0.25)
                    Q3 = data.quantile(0.75)
                    IQR = Q3 - Q1
                    filtered_data = data[(data >= Q1 - 1.5 * IQR) & (data <= Q3 + 1.5 * IQR)]
                    
                    # Create histogram
                    ax.hist(filtered_data, bins=30, alpha=0.7, color=colors[i % len(colors)], edgecolor='black')
                    ax.set_title(f'{col}\n(n={len(filtered_data):,})', fontweight='bold')
                    ax.set_xlabel('Value')
                    ax.set_ylabel('Frequency')
                    ax.grid(True, alpha=0.3)
                    
                    # Add statistics text
                    mean_val = filtered_data.mean()
                    std_val = filtered_data.std()
                    ax.axvline(mean_val, color='red', linestyle='--', alpha=0.8, label=f'Mean: {mean_val:.2f}')
                    ax.legend()
        
        # Remove empty subplots
        for i in range(len(main_cols), len(axes)):
            fig.delaxes(axes[i])
        
        plt.suptitle(f'📊 Performance Metrics Distribution - {main_table}', fontsize=16, fontweight='bold')
        plt.tight_layout()
        plt.show()
else:
    print("⚠️ No performance metrics columns identified.")

## 7. Time Series Analysis

Analyze temporal patterns in the data.

In [ ]:
# Find timestamp columns
timestamp_cols = ['last_update', 'timestamp', 'date', 'time']
time_data = {}

for table_name, df in sample_data.items():
    for col in timestamp_cols:
        if col in df.columns:
            try:
                # Convert to datetime
                df[col] = pd.to_datetime(df[col], errors='coerce')
                time_data[table_name] = {'df': df, 'time_col': col}
                print(f"✅ Found timestamp column '{col}' in {table_name}")
                break
            except Exception as e:
                print(f"❌ Error processing {col} in {table_name}: {e}")

print(f"\n⏰ Found time data in {len(time_data)} tables")

In [ ]:
# Time series visualization
if time_data:
    fig, axes = plt.subplots(len(time_data), 1, figsize=(15, 6 * len(time_data)))
    if len(time_data) == 1:
        axes = [axes]
    
    for i, (table_name, data) in enumerate(time_data.items()):
        df = data['df']
        time_col = data['time_col']
        ax = axes[i]
        
        # Create time series plot
        df_clean = df.dropna(subset=[time_col])
        
        if len(df_clean) > 0:
            # Group by hour and count records
            df_clean['hour'] = df_clean[time_col].dt.hour
            hourly_counts = df_clean.groupby('hour').size()
            
            ax.plot(hourly_counts.index, hourly_counts.values, marker='o', linewidth=2, markersize=6)
            ax.set_title(f'📅 Hourly Data Distribution - {table_name}', fontweight='bold')
            ax.set_xlabel('Hour of Day')
            ax.set_ylabel('Number of Records')
            ax.grid(True, alpha=0.3)
            ax.set_xticks(range(0, 24, 2))
            
            # Add date range info
            min_date = df_clean[time_col].min()
            max_date = df_clean[time_col].max()
            ax.text(0.02, 0.98, f'Date Range: {min_date.strftime("%Y-%m-%d")} to {max_date.strftime("%Y-%m-%d")}',
                   transform=ax.transAxes, verticalalignment='top',
                   bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.8))
    
    plt.tight_layout()
    plt.show()
else:
    print("⚠️ No timestamp columns found in the data.")

## 8. Correlation Analysis

Analyze correlations between different metrics.

In [ ]:
# Correlation analysis for tables with multiple numeric columns
for table_name, df in sample_data.items():
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    
    if len(numeric_cols) >= 3:  # Need at least 3 columns for meaningful correlation
        print(f"\n🔗 Correlation Analysis for {table_name}:")
        
        # Calculate correlation matrix
        corr_matrix = df[numeric_cols].corr()
        
        # Create correlation heatmap
        plt.figure(figsize=(12, 10))
        mask = np.triu(np.ones_like(corr_matrix, dtype=bool))  # Mask upper triangle
        
        sns.heatmap(corr_matrix, 
                   mask=mask,
                   annot=True, 
                   cmap='RdYlBu_r', 
                   center=0,
                   square=True,
                   linewidths=0.5,
                   fmt='.2f',
                   cbar_kws={"shrink": .8})
        
        plt.title(f'🔥 Correlation Heatmap - {table_name}', fontsize=14, fontweight='bold')
        plt.tight_layout()
        plt.show()
        
        # Find strong correlations (> 0.7 or < -0.7)
        strong_corr = []
        for i in range(len(corr_matrix.columns)):
            for j in range(i+1, len(corr_matrix.columns)):
                corr_val = corr_matrix.iloc[i, j]
                if abs(corr_val) > 0.7:
                    strong_corr.append({
                        'Variable 1': corr_matrix.columns[i],
                        'Variable 2': corr_matrix.columns[j],
                        'Correlation': corr_val
                    })
        
        if strong_corr:
            strong_corr_df = pd.DataFrame(strong_corr).sort_values('Correlation', key=abs, ascending=False)
            print("\n💪 Strong Correlations (|r| > 0.7):")
            display(strong_corr_df)
        else:
            print("\n📝 No strong correlations found (|r| > 0.7)")
        
        break  # Only show first table to avoid clutter

## 9. Geographic and Regional Analysis

Analyze data distribution by geographic regions.

In [ ]:
# Find geographic columns
geo_keywords = ['region', 'kabupaten', 'city', 'province', 'area', 'location', 'lat', 'lon', 'coordinate']
geo_data = {}

for table_name, df in sample_data.items():
    geo_cols = [col for col in df.columns if any(keyword in col.lower() for keyword in geo_keywords)]
    if geo_cols:
        geo_data[table_name] = {'df': df, 'geo_cols': geo_cols}

print(f"🌍 Found geographic data in {len(geo_data)} tables:")
for table, data in geo_data.items():
    print(f"   {table}: {data['geo_cols']}")

In [ ]:
# Geographic distribution analysis
if geo_data:
    for table_name, data in geo_data.items():
        df = data['df']
        geo_cols = data['geo_cols']
        
        print(f"\n🗺️ Geographic Analysis for {table_name}:")
        
        # Analyze each geographic column
        for col in geo_cols[:2]:  # Limit to first 2 columns
            if df[col].dtype == 'object' or df[col].dtype.name == 'category':
                # Categorical geographic data
                value_counts = df[col].value_counts().head(10)
                
                if len(value_counts) > 1:
                    plt.figure(figsize=(12, 6))
                    
                    # Bar plot
                    bars = plt.bar(range(len(value_counts)), value_counts.values, color=colors[:len(value_counts)])
                    plt.title(f'📍 Distribution by {col} - {table_name}', fontweight='bold')
                    plt.xlabel(col.title())
                    plt.ylabel('Number of Records')
                    plt.xticks(range(len(value_counts)), value_counts.index, rotation=45, ha='right')
                    
                    # Add value labels
                    for bar, value in zip(bars, value_counts.values):
                        plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01*max(value_counts.values),
                                f'{value:,}', ha='center', va='bottom')
                    
                    plt.tight_layout()
                    plt.show()
                    
                    print(f"\n📊 Top 10 {col} values:")
                    for location, count in value_counts.items():
                        print(f"   {location}: {count:,} records")
        
        break  # Only show first table
else:
    print("⚠️ No geographic columns identified.")

## 10. Data Quality Assessment

Assess data quality including missing values, duplicates, and outliers.

In [ ]:
# Comprehensive data quality analysis
quality_report = {}

for table_name, df in sample_data.items():
    print(f"\n🔍 Data Quality Assessment for {table_name}:")
    
    # Basic statistics
    total_rows = len(df)
    total_cols = len(df.columns)
    
    # Missing values
    missing_counts = df.isnull().sum()
    missing_percentages = (missing_counts / total_rows) * 100
    
    # Duplicate rows
    duplicate_rows = df.duplicated().sum()
    duplicate_percentage = (duplicate_rows / total_rows) * 100
    
    # Columns with all missing values
    empty_cols = missing_counts[missing_counts == total_rows].index.tolist()
    
    # Numeric columns with potential outliers
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    outlier_cols = []
    
    for col in numeric_cols:
        if df[col].notna().sum() > 10:  # At least 10 non-null values
            Q1 = df[col].quantile(0.25)
            Q3 = df[col].quantile(0.75)
            IQR = Q3 - Q1
            outliers = df[(df[col] < Q1 - 1.5 * IQR) | (df[col] > Q3 + 1.5 * IQR)][col]
            if len(outliers) > 0:
                outlier_cols.append({'column': col, 'outlier_count': len(outliers), 'outlier_pct': len(outliers)/len(df)*100})
    
    # Store quality metrics
    quality_report[table_name] = {
        'total_rows': total_rows,
        'total_cols': total_cols,
        'duplicate_rows': duplicate_rows,
        'duplicate_pct': duplicate_percentage,
        'empty_columns': len(empty_cols),
        'columns_with_missing': len(missing_counts[missing_counts > 0]),
        'outlier_columns': len(outlier_cols)
    }
    
    print(f"   📏 Dimensions: {total_rows:,} rows × {total_cols} columns")
    print(f"   🔄 Duplicate rows: {duplicate_rows:,} ({duplicate_percentage:.1f}%)")
    print(f"   🕳️ Empty columns: {len(empty_cols)}")
    print(f"   ❓ Columns with missing data: {len(missing_counts[missing_counts > 0])}/{total_cols}")
    print(f"   🎯 Columns with outliers: {len(outlier_cols)}")
    
    # Show missing data details
    high_missing = missing_percentages[missing_percentages > 10].sort_values(ascending=False)
    if not high_missing.empty:
        print(f"\n   ⚠️ Columns with >10% missing data:")
        for col, pct in high_missing.head(5).items():
            print(f"     • {col}: {pct:.1f}% missing")

# Create quality summary visualization
if quality_report:
    quality_df = pd.DataFrame(quality_report).T
    
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    
    # Duplicate percentages
    axes[0,0].bar(quality_df.index, quality_df['duplicate_pct'], color='lightcoral')
    axes[0,0].set_title('🔄 Duplicate Rows (%)', fontweight='bold')
    axes[0,0].set_ylabel('Percentage')
    axes[0,0].tick_params(axis='x', rotation=45)
    
    # Columns with missing data
    axes[0,1].bar(quality_df.index, quality_df['columns_with_missing'], color='lightblue')
    axes[0,1].set_title('❓ Columns with Missing Data', fontweight='bold')
    axes[0,1].set_ylabel('Count')
    axes[0,1].tick_params(axis='x', rotation=45)
    
    # Empty columns
    axes[1,0].bar(quality_df.index, quality_df['empty_columns'], color='lightyellow')
    axes[1,0].set_title('🕳️ Completely Empty Columns', fontweight='bold')
    axes[1,0].set_ylabel('Count')
    axes[1,0].tick_params(axis='x', rotation=45)
    
    # Outlier columns
    axes[1,1].bar(quality_df.index, quality_df['outlier_columns'], color='lightgreen')
    axes[1,1].set_title('🎯 Columns with Outliers', fontweight='bold')
    axes[1,1].set_ylabel('Count')
    axes[1,1].tick_params(axis='x', rotation=45)
    
    plt.suptitle('📊 Data Quality Assessment Summary', fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.show()

## 11. Interactive Plotly Visualizations

Create interactive charts for deeper exploration.

In [ ]:
# Create interactive visualizations with Plotly
if sample_data:
    # Select main table for interactive analysis
    main_table = list(sample_data.keys())[0]
    main_df = sample_data[main_table]
    
    # Get numeric columns
    numeric_cols = main_df.select_dtypes(include=[np.number]).columns.tolist()
    
    if len(numeric_cols) >= 2:
        print(f"🎮 Creating interactive visualizations for {main_table}...")
        
        # Interactive scatter plot
        if len(numeric_cols) >= 2:
            col1, col2 = numeric_cols[0], numeric_cols[1]
            
            # Sample data for performance
            plot_df = main_df[[col1, col2]].dropna().sample(min(1000, len(main_df)))
            
            fig = px.scatter(plot_df, x=col1, y=col2,
                           title=f'🔍 Interactive Scatter Plot: {col1} vs {col2}',
                           labels={col1: col1.title(), col2: col2.title()},
                           template='plotly_white')
            
            fig.update_traces(marker=dict(size=6, opacity=0.6))
            fig.update_layout(height=500)
            fig.show()
        
        # Interactive histogram
        if len(numeric_cols) >= 1:
            col = numeric_cols[0]
            data = main_df[col].dropna()
            
            # Remove extreme outliers for better visualization
            Q1 = data.quantile(0.05)
            Q3 = data.quantile(0.95)
            filtered_data = data[(data >= Q1) & (data <= Q3)]
            
            fig = px.histogram(x=filtered_data, nbins=50,
                             title=f'📊 Interactive Histogram: {col}',
                             labels={'x': col.title(), 'y': 'Frequency'},
                             template='plotly_white')
            
            fig.update_layout(height=400)
            fig.show()
    
    else:
        print("⚠️ Not enough numeric columns for interactive plots.")

## 12. Export and Summary Report

Generate a comprehensive summary and save visualizations.

In [ ]:
# Create comprehensive summary report
print("📋 COMPREHENSIVE RAN DATABASE ANALYSIS REPORT")
print("=" * 60)

# Database overview
total_tables = len(tables)
total_sample_rows = sum(len(df) for df in sample_data.values())
total_columns = sum(len(df.columns) for df in sample_data.values())

print(f"\n🏗️ DATABASE OVERVIEW:")
print(f"   • Total Tables: {total_tables}")
print(f"   • Sample Rows Analyzed: {total_sample_rows:,}")
print(f"   • Total Columns: {total_columns}")
print(f"   • Database Size: {DATABASE_PATH.stat().st_size / (1024*1024):.1f} MB")

# Technology distribution
print(f"\n📡 TECHNOLOGY DISTRIBUTION:")
for tech, count in tech_distribution.items():
    print(f"   • {tech}: {count:,} records ({count/sum(tech_distribution.values())*100:.1f}%)")

# Data quality summary
if quality_report:
    print(f"\n🔍 DATA QUALITY SUMMARY:")
    avg_duplicates = np.mean([report['duplicate_pct'] for report in quality_report.values()])
    total_empty_cols = sum([report['empty_columns'] for report in quality_report.values()])
    total_outlier_cols = sum([report['outlier_columns'] for report in quality_report.values()])
    
    print(f"   • Average Duplicate Rate: {avg_duplicates:.1f}%")
    print(f"   • Total Empty Columns: {total_empty_cols}")
    print(f"   • Columns with Outliers: {total_outlier_cols}")

# Performance metrics found
if perf_columns:
    total_perf_metrics = sum(len(cols) for cols in perf_columns.values())
    print(f"\n🎯 PERFORMANCE METRICS:")
    print(f"   • Tables with Performance Data: {len(perf_columns)}")
    print(f"   • Total Performance Metrics: {total_perf_metrics}")

# Geographic coverage
if geo_data:
    print(f"\n🌍 GEOGRAPHIC COVERAGE:")
    print(f"   • Tables with Geographic Data: {len(geo_data)}")
    for table, data in geo_data.items():
        print(f"   • {table}: {len(data['geo_cols'])} geographic columns")

# Time coverage
if time_data:
    print(f"\n⏰ TIME COVERAGE:")
    for table, data in time_data.items():
        df = data['df']
        time_col = data['time_col']
        df_clean = df.dropna(subset=[time_col])
        if len(df_clean) > 0:
            min_date = df_clean[time_col].min()
            max_date = df_clean[time_col].max()
            print(f"   • {table}: {min_date.strftime('%Y-%m-%d')} to {max_date.strftime('%Y-%m-%d')}")

print(f"\n✅ ANALYSIS COMPLETE!")
print(f"📊 Generated visualizations include:")
print(f"   • Technology distribution charts")
print(f"   • Performance metrics histograms")
print(f"   • Time series analysis")
print(f"   • Correlation heatmaps")
print(f"   • Geographic distribution plots")
print(f"   • Data quality assessments")
print(f"   • Interactive Plotly visualizations")

print(f"\n🚀 NEXT STEPS:")
print(f"   1. Use insights for NER model training")
print(f"   2. Develop SQL query templates based on schema")
print(f"   3. Create performance benchmarks")
print(f"   4. Build automated monitoring dashboards")

In [ ]:
# Save analysis results (optional)
output_dir = PROJECT_ROOT / "data" / "processed" / "analysis_results"
output_dir.mkdir(parents=True, exist_ok=True)

# Save table summary
if 'summary_df' in locals():
    summary_df.to_csv(output_dir / "table_summary.csv", index=False)
    print(f"💾 Table summary saved to: {output_dir / 'table_summary.csv'}")

# Save quality report
if quality_report:
    quality_df = pd.DataFrame(quality_report).T
    quality_df.to_csv(output_dir / "data_quality_report.csv")
    print(f"💾 Quality report saved to: {output_dir / 'data_quality_report.csv'}")

print(f"\n🎉 Analysis complete! All results available in the notebook and saved files.")